<a href="https://colab.research.google.com/github/mzhkz/220630-Dapps-handson/blob/completed/0904_detect_kana_yomi_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備
## 必要なパッケージ等を読み込み/割り当て

In [ ]:
! pip install mecab-python3 unidic-lite
! pip install plyvel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.3 MB/s 
     |████████████████████████████████| 47.4 MB 189 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=b74c2e006d8a9d6b07882619d065169066ae21c7bf52301e46d04b14d6bbd1be
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 693 kB 5.2 MB/s 


In [ ]:
import numpy as np
import requests as reqs
import json
import regex
import plyvel
import pickle
import MeCab

## 作業ディレクトリ（ファイルの読み込みや保存を行う絶対パス）

In [ ]:
APP_WORKDIR = "/content/drive/MyDrive/coefont_kana_converter_error_detector/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### APIのアクセストークンなど機密情報の管理

In [ ]:
secrets = {}
with open(APP_WORKDIR + 'secrets.json') as f:
  secrets = json.load(f)

### データベースのセットアップ（品詞の保存）
* 読み書きの早い、Key -> Value型のローカルファイル型データベースをである。
* Ethereumのノードの内部ではトランザクションの管理とかでも使われている。
* https://github.com/google/leveldb
* https://plyvel.readthedocs.io/en/latest/

In [ ]:
vocab_db = plyvel.DB(APP_WORKDIR + 'vocabs.ldb', create_if_missing=True)
lebels_db = plyvel.DB(APP_WORKDIR + 'lebels.ldb', create_if_missing=True)

dbs = [vocab_db, lebels_db]

### パスを間違えたりして再読み込みする場合は以下セルを実行してから上のセルを実行する

In [ ]:
for db in dbs:
  db.close()

## 各種依存記号の設定

In [ ]:
SYMBOL_READING_POINT = "、" # 句読点
SYMBOL_PUNCTUATION = "。" # 読点
SYMBOL_HALF_SPACE = " " # 半角スペース
SYMBOL_LONG_NOTE = "ー" # 伸ばし棒
SYMBOL_NONE = "" # 空文字
SYMBOL_TAB = "\t" # TAB
SYMBOL_SEMICORON = ";" # SEMICORON

# データソース別のクラス
* 単語中抽出に必要な対象（Webページなど）を管理する
* 名詞の抽出とMeCabによるよみラベルを管理する。

## 要件
*   Webサイトからテキストをスクレイピング（Webサイトごとにテンプレートを作成する）
*   リンクや特殊記号を排除する
*   文章の形態素解析を行い、漢字が含まれている名詞のみを抽出する。



### MecabのWakatiで初期化

In [ ]:
wakati = MeCab.Tagger()

### 漢字判定用の正規表現

In [ ]:
KANJI_REG_PETERN = regex.compile(r'\p{Script=Han}+')

## ローダー

In [ ]:
class DataSource:
  def __init__(self):
    self.vocabs = []
    self.labels = [] 

  def _fetchData(self): # 文章（テキストデータ）を読み込む、ここは各サービスごとに取得の方法が異なるためオーバーライドする。
    pass

  def preprosessing(self, subject):
    handled = self._replaceSymbol(subject) #ここから文字列
    return handled

  def _replaceSymbol(self, subject):
    sentence = subject.replace(SYMBOL_TAB, SYMBOL_NONE)
    return sentence

  def load(self):
    text = self._fetchData()
    text = self.preprosessing(text)
    vocabs, labels = self._analysis(text)
    self.vocabs = vocabs
    self.labels = labels 

  def _analysis(self, subject):
    result = wakati.parse(subject) # 形態素解析分析
    vocabs = [line.split(SYMBOL_TAB) for line in result.splitlines()]
    nouns = [vocab_data for vocab_data in vocabs if len(vocab_data) >= 4 and "名詞" in vocab_data[4][0:2] and KANJI_REG_PETERN.search(vocab_data[0])] # 漢字を含む名詞のみ抽出
    return [noun[0] for noun in nouns],  [noun[2] for noun in nouns] # 名詞とMeCabによるかな変換を取得

  def save(self, new_count=False):
    reg_vocab_count = 0
    for vocab, label in zip(self.vocabs, self.labels):
      binary_key = vocab.encode("utf-8") # key for level db
      binary_lebel = label.encode("utf-8") # key for level db
      if new_count and vocab_db.get(binary_key) is None:
        reg_vocab_count +=1
      vocab_db.put(binary_key, binary_lebel)
    return reg_vocab_count

## 各種サービスなど、オケージョンごとのローダー

ローダーの対象
*   note.com （記事サービス）
*   wikipedia.org (辞書)
*   ローカルファイル
*   変数



### 変数からテキストを読み込む

In [ ]:
class ArgDataSource(DataSource):
  def __init__(self, source):
    super().__init__()
    self.source = source

  def _fetchData(self):
    return self.source

### ローカルのテキストファイルからテキストを読み込む（バイナリファイル未対応）

In [ ]:
class LocalDataSource(DataSource):
  def __init__(self, file_path):
    super().__init__()
    self.file_path = file_path

  def _fetchData(self):
    with open(self.file_path) as f:
      lines = f.readlines()
      self.source = "".join(lines)
    return self.source

### Note API v2
note.comから記事を取得する

In [ ]:
class NoteDataSource(DataSource):
  def __init__(self, note_article_key):
    super().__init__()
    self.note_article_key = note_article_key

  def _fetchData(self):
    url = "https://note.com/api/v1/notes/{}".format(self.note_article_key)
    response = reqs.get(url=url)
    text = response.text
    response_json = json.loads(text)
    return response_json["data"]["body"]

### Wikipedia

https://ja.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles=%E6%85%B6%E6%87%89%E7%BE%A9%E5%A1%BE%E5%A4%A7%E5%AD%A6

In [ ]:
class WikipediaDataSource(DataSource):
  def __init__(self, title):
    super().__init__()
    self.title = title

  def _fetchData(self):
    url = "https://ja.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1"
    params = {
        "titles": self.title,
    }
    response = reqs.get(url=url, params=params)
    text = response.text
    response_json = json.loads(text)
    return list(response_json["query"]["pages"].values())[0]["extract"]

## 各種ローダーのテスト

In [ ]:
test01_note_datasource = NoteDataSource(note_article_key="n4ece27ed112b")
test01_note_datasource.load()

print(test01_note_datasource.vocabs)
print(test01_note_datasource.labels)

test01_note_datasource.save(new_count=True)

['大学', '入学', '時代', '人間', '選択', '幸せ', '伝播', '自分', '不安', '勉強', '自負', '技術', '修得', '人', '一', '倍', '熱量', '努力', '実際', '学生', '身分', '個人', '開発', '仕事', '社会', '感覚', '自身', '大学', '生活', '半ば', '先生', '方', '言葉', '魂', '所属', '研究', '会', '大学', '友人', '魂', '魂', '価値', '方向', '魂', '言語', '自分', '学び', '本質', '事実', '記事', '魂', '言語', '今後', '学び', '記録', '執筆', '実際', '魂', '自分', '迷走', '逆', '自分', '言葉', '勉強', '自分', '言葉', '体得', '自分', '言葉', '宝物', '月', '終わり', '友人', '勢い', '文中', '箇所', '筆者', '教え', '記事', '理解', '問題', '発見', '解決', '最近', '以外', '問題', '発見', '解決', '中等', '教育', '学校', '反面', '言葉', '本質', '事例', '問題', '発見', '解決', '教育', '時代', '背景', '論点', '整理', '視点', '在学', '骨', '髄', '視点', '変容', '個人', '一', '意見', '参考', '学び', '人', '本当', '少数', '自身', '解釈', '最近', '言語', '学び', '考え', '解釈', '選択', '注意', '公式', '見解', '一切', '自身', '独断', '偏見', '構成', '解釈', '個人', '主張', '念頭', '進学', '意図', '特徴', '理系', '文系', '区別', '自由', '分野', '横断', '学び', '意見', '内外', '耳', '本質', '着眼', '点', '縦割り', '問題', '意識', '背景', '必要', '見解', '縦割り', '対応', '世界', '帰着', '複雑', '問題', '解決', '議論', '解決', '担保', '

0

In [ ]:
test01_wiki_datasource = WikipediaDataSource(title="慶應義塾大学")
test01_wiki_datasource.load()

print(test01_wiki_datasource.vocabs)
print(test01_wiki_datasource.labels)

test01_wiki_datasource.save(new_count=True)

['慶應', '義塾', '大学', '英語', '東京', '都', '港', '区', '三田', '丁目', '番', '号', '本部', '日本', '私立', '大学', '年', '創立', '年', '大学', '設置', '大学', '略称', '慶應', '慶大', '應', '旧', '字体', '報道', '慶応', '表記']
['ケイオウ', 'ギジュク', 'ダイガク', 'エイゴ', 'トウキョウ', 'ト', 'ミナト', 'ク', 'ミタ', 'チョウメ', 'バン', 'ゴウ', 'ホンブ', 'ニッポン', 'シリツ', 'ダイガク', 'ネン', 'ソウリツ', 'ネン', 'ダイガク', 'セッチ', 'ダイガク', 'リャクショウ', 'ケイオウ', 'ケイダイ', '應', 'キュウ', 'ジタイ', 'ホウドウ', 'ケイオウ', 'ヒョウキ']


24

# コンバーターのクラス

* 漢字をかな変換するコンバータ
* クラスとして機能を丸めることで複数のコンバーターの差異を吸収する。コンバーターを適用する順序を入れ替えるなど

In [ ]:
class BaseConverter:
  def __init__(self, conveter_name):
    self.converter_name = conveter_name
    self.split_code = SYMBOL_SEMICORON

  def preprosessing(self, subject):
    return subject

  def convert(self, vocabs):
    print(vocabs)
    sentence = SYMBOL_SEMICORON.join(vocabs)
    result = self._execute_api(sentence)
    converted_vocabs = [self.preprosessing(vocab) for vocab in result.split(SYMBOL_SEMICORON)]
    print(converted_vocabs)
    return converted_vocabs

  def get_indexkey(self, noun):
    return "{}_{}".format(self.converter_name, noun).encode("utf-8")

## CoeFont API (target)
* CoefontのAPIのアクセス方法とアクセスキーが変わり次第、実装する

In [ ]:
SYMBOL_TRIGGER_1 = [
        ["オ", "コ", "ソ", "ト", "ノ", "ホ", "モ", "ヨ", "ロ", "ヲ"],
        ["エ", "ケ", "セ", "ネ", "ヘ", "メ", "レ"], # removed テ
        ["ア", "カ", "サ", "タ", "ナ", "ハ", "マ", "ラ", "ヤ", "ラ", "ワ"]
        ]

SYMBOL_TRIGGER_2 = [
        ["ウ"],
        ["イ"],
        ["ア"]
        ]

class CoeFontConverter(BaseConverter):
  def __init__(self, api_key):
    super().__init__("coefont")
    self.api_key = api_key

  def _execute_api(self, sentence):
    # coefontの実装
    pass

  def preprosessing(self, subject):
    list_kat_subject = list(subject)
    kat_subject_size = len(list_kat_subject)-1
    idkc = 0
    while  idkc < kat_subject_size:
      kat_char_pointer = list_kat_subject[idkc]
      kat_char_next = list_kat_subject[idkc+1]
      if kat_char_next == SYMBOL_LONG_NOTE:
         for ids, symbols in enumerate(SYMBOL_TRIGGER_1):
           if kat_char_pointer in symbols:
               list_kat_subject[idkc+1] = SYMBOL_TRIGGER_2[ids][0]
               idkc+=1
      idkc+=1
    return "".join(list_kat_subject)

## Goo API
* かな変換APIを使用する。
* ドキュメントはこちら
  * https://labs.goo.ne.jp/api/jp/hiragana-translation/

In [ ]:
class GooConverter(BaseConverter):
  def __init__(self, app_id):
    super().__init__("goo")
    self.app_id = app_id

  def _execute_api(self, sentence):
    url = "https://labs.goo.ne.jp/api/hiragana"
    params = {
        "app_id": self.app_id,
        "sentence": sentence,
        "output_type": "katakana"
    }

    headers = {'Content-Type': 'application/json'}

    r = reqs.post(url=url, data=params)
    response_text = r.text
    print(response_text)
    response_json = json.loads(response_text)
    return response_json["converted"]

  def preprosessing(self, subject):
     return subject.replace(SYMBOL_HALF_SPACE, SYMBOL_NONE)

# データソース毎に読みの検証を行う
* コンバータを用いてCoeFontの単語出力と比較を行う。
* 比較の結果、不一致だった場合は他のコンバーターにおいても比較を行い、正解ラベルを付与できるよう分布を収束させる。

In [ ]:
class ConverterCompetition:
  def __init__(self, converters):
    self.converters = converters

  def _convert_all_on(self, data_source, converter_id):
    vocabs = data_source.vocabs
    converter = self.converters[converter_id]

    db_keys = [converter.get_indexkey(vocab) for vocab in vocabs]
    caches = [lebels_db.get(db_key) for db_key in db_keys] # キャッシュがあったら1 なければ 0
    entred_vocab = [vocab for idv, vocab in enumerate(vocabs) if caches[idv] is None] # キャッシュがなければあたらしくしゅとくする

    new_yomi_labels = []
    if (len(entred_vocab) > 0):
      new_yomi_labels = converter.convert(entred_vocab)

    for vocab, label in zip(entred_vocab, new_yomi_labels):
      lebels_db.put(converter.get_indexkey(vocab), label.encode("utf-8")) #　データベースに新しい単語を保存

    yomi_labels = []
    iter_new_yomi_labels = iter(new_yomi_labels)
    for idc, cache in enumerate(caches):
      if cache:
        yomi_labels.append(cache.decode("utf-8")) # キャッシュ（db）にある場合はキャッシュから取得
      else:
        yomi_labels.append(next(iter_new_yomi_labels)) # ない場合は、先ほど変換した結果から取得
    
    return vocabs, yomi_labels

  def compete(self, data_source, converter_id=0):
    vocabs, yomi_labels = self._convert_all_on(data_source, converter_id)
    errors = []
    for idn, noun in enumerate(vocabs):
      yomi_label = yomi_labels[idn]
      mecab_label = data_source.labels[idn]

      if yomi_label != mecab_label:
         errors.append([noun, yomi_label, mecab_label])
    return errors

In [25]:
test_04_converters = [CoeFontConverter(api_key=""), GooConverter(app_id=secrets["goo_api_id"])]
test_04_competition = ConverterCompetition(converters=test_04_converters)

test_04_competition.compete(test01_note_datasource,converter_id=1) # noteの記事をデータソースにgooAPIを検証する。

[['月', 'ツキ', 'ガツ'],
 ['一', 'イチ', 'ヒト'],
 ['一', 'イチ', 'ヒト'],
 ['一', 'イチ', 'ヒト'],
 ['他', 'ホカ', 'タ'],
 ['他', 'ホカ', 'タ'],
 ['体', 'カラダ', 'タイ'],
 ['二', 'ニー', 'ニ']]

# データソースを自動的に取得する
* データソースを主導で作成するのではなく、永続的に新しいデータソースから単語を取得できるエコシステムを構築する

* WebhookやWebsoket, Server-sent Eventなどを用いてデータソースを自動で収集する
(Wikipediaのstreaming機能など)

## 要件
*   永続的にプログラムがデータソースを自動で生成する
*   単語のバリエーションの広がりを担保する








## 案1: Wikipedia Streaming

*   SSEにかけて、日本語のwikipediaの更新ログをとる
*   WikipediaDataSourceにかけて、単語と第一レイヤーの読み予測ラベル（MeCab）を取得する。
*   単語ごとに過去にCoeFontで検証したかどうかを問い合わせる。以下の条件を満たした場合は単語は次へ
  * キーが「subject_noun」のvalueがNoneかどうか: Coefontの結果をすでに持っているか
  * キーが「test_noun」のvalueがNoneかどうか: 検証が既に完了済みの名詞かどうか
*   Coefontへ品詞のよみ変換をAPI経由で取得する。（いくつかの未検証の単語をまとめてバッチ処理）
*   第一レイヤーの読み予測ラベルとCoeFontの読みが一致しなかった場合、第二レイヤーのコンバーターで比較を行う。
*   過半数以上のよみを正解ラベルとしてデータベースおよびSpreadsheetに記録
*  **これらを再帰的に実行**

# 実行